In [ ]:
# Group : Groupe B2-4 [Equipe 8]
# AI name : Greedy preprocessing 
# Members : Jules Peignier, Aymen Kallala 
# Description : Greedy Prep est un greedy de base où le chemin est calculé dans le preprocessing

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'
 
 
# Librarie containing FIFO structure
 
import heapq

In [ ]:
def push_to_structure (element:tuple,priority_queue) :
  heapq.heappush(priority_queue,element)

def pop_from_structure (priority_queue) :
   return heapq.heappop(priority_queue)

# Implementation of BFS in order to create a routing table 
    
def traversal (start_vertex:tuple, graph:dict) :

    priority_queue = []
    distance={}
    #initialitation à l'infini des distances
    for elt in graph:
      distance[elt]=10**50

    # Add the starting vertex with None as parent
    push_to_structure((0,start_vertex),priority_queue) 
 
    # Initialize the outputs
    current_vertex=start_vertex
    explored_vertices = [] 
    routing_table = {} 

    distance[current_vertex]=0
    routing_table[current_vertex]=None
    
    # Iterate while some vertices remain
    while priority_queue!=[] :
        
        current_distance,current_vertex=pop_from_structure(priority_queue)
        
        explored_vertices.append(current_vertex)
            
        for neighbor in graph[current_vertex]:
          if neighbor not in explored_vertices:
           
            push_to_structure((current_distance+graph[current_vertex][neighbor],neighbor),priority_queue)
          
          if current_distance+graph[current_vertex][neighbor] < distance[neighbor] :
           
            distance[neighbor] = current_distance+graph[current_vertex][neighbor]
           
            routing_table[neighbor] = current_vertex
                    
    return routing_table,distance
 
# Algorithm providing the shortest path between starting location and the piece of cheese

In [ ]:
def build_meta_graph (maze_map, locations) :
  routing_table_cheeses={}
  meta_graph={ fromage : {} for fromage in locations}
  for start_fromage in locations : 
    routing_table, distance = traversal(start_fromage , maze_map )
    routing_table_cheeses[start_fromage] = routing_table
    for target_fromage in locations :
      meta_graph[start_fromage][target_fromage] = distance[target_fromage]
  return meta_graph, routing_table_cheeses
#return meta_graph,routing_table_cheeses
    # Return the meta-graph and all necessary routing tables

In [ ]:
def griddy(current_vertex, meta_graph, path):
  best_length=10**5
  for fromages in meta_graph :
    if fromages not in path :
      if meta_graph[current_vertex][fromages] < best_length :
        best_length =  meta_graph[current_vertex][fromages]
        best_fromage = fromages
  return best_fromage

In [ ]:
def implementation_griddy(current_vertex, meta_graph):
  path=[current_vertex]
  for i in range(len(meta_graph)-1):
    fromage=griddy(current_vertex, meta_graph, path)
    path.append(fromage)
    current_vertex = fromage
  return path


In [ ]:
def moves_from_locations (locations) :
    # Transform a series of locations into corresponding moves to realize it
  list_moves=[]
  k=len(locations)-1
  i=0
  while k!=0 :
    if locations[i]==locations[i+1] :
      k-=2
      locations.pop(i+1)
    else :
      k-=1
    list_moves.append(move_from_locations(locations[i],locations[i+1]))
    i+=1
  return list_moves

In [ ]:
#détermine un mouvement pour deux vecteurs voisins 
def move_from_locations (source_location, target_location) : 
    difference = (target_location[0] - source_location[0], target_location[1] - source_location[1])
    if difference == (0, -1) :
        return MOVE_DOWN
    elif difference == (0, 1) :
        return MOVE_UP
    elif difference == (1, 0) :
        return MOVE_RIGHT
    elif difference == (-1, 0) :
        return MOVE_LEFT


In [ ]:
# Use the routing table to find the sequence of locations from source to target
def find_route (routing_table, source_location, target_location) :
  path=[target_location] #On initialise une liste de points en commencant par la fin
  current_vertex=target_location
  while source_location!=current_vertex:
    #tant que le point de départ n'est pas désigné comme destination
    parent=routing_table[current_vertex]
    #on ajoute les parents à l'itinéraire
    path.append(parent)
    # le parent devient le vertex de destination
    current_vertex=parent
  reversed_path = []
 
  while path != [] :
      reversed_path.append(path.pop())
  return reversed_path
  #on retourne la liste pour avoir l'itinéraire du départ vers l'arrivée

In [ ]:
def meta_graph_route_to_route (meta_graph_route, routing_tables) :
  final_path=[]
  for i in range (len(meta_graph_route)-1):
    final_path+=find_route (routing_tables[meta_graph_route[i]], meta_graph_route[i], meta_graph_route[i+1])
  return final_path
    # Return the sequence of locations in the maze to perform a route in the meta-graph

In [ ]:
list_moves=[]
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :
       # Implement the list of moves to perform in the preprocess
    global list_moves
    #list_moves = moves_from_locations(find_route(traversal(player_location,maze_map)[0],player_location,pieces_of_cheese[0]))
   
    meta_graph, routing_table_cheeses = build_meta_graph (maze_map, [player_location]+pieces_of_cheese)
    list_moves = moves_from_locations(meta_graph_route_to_route (implementation_griddy(player_location, meta_graph) , routing_table_cheeses))

In [ ]:
#turn va renvoyer le dernier move de la liste "list_moves" à chaque appel
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
    global list_moves
   # print(pieces_of_cheese)
    if len(list_moves) > 0 :
       return list_moves.pop(0) #on extrait le premier élément et on le retourne  
    else :
      Exception ("Pas de Mouvement")  
      #raise Exception ("Pas de Mouvement")